In [ ]:
ls ../data

In [2]:
import sys
import os
sys.path.append('../code')
from io import StringIO
from subprocess import Popen, PIPE, run
from HDR_run import HDR_master

home = '/Users/mahtin'
path = '/Users/mahtin/Dropbox/Icke/Work/somVar/HDRtest'
bam_file = f'{path}/data/002.bam '
chrom = 'chr1'
# chrom = ''
pileup_file = f'{path}/data/002.pileup'
threads = 8
mut_file = f'{path}/data/002.csv'
genome_split_path = os.path.join(home,'Dropbox/Icke/Work/static/genome/gatk/hg38/split')
HDR_config = {
    "minAltSum": 2,
    "minAltRatio": 0.1,
    "maxAltRatio": 0.85,
    "MINQ": 15,
    "MINSIM": .50,
    "PAD": 100,
    "MINq": 10,
    "MinAltSupport": 5,
    "MinHDRCount": 1,
    "pile2hotspot": '../shell/pile2hotspot.mawk',#
    "pile2hotspot_chrom": '../shell/pile2hotspot_chrom.mawk',
    "editbam": '../shell/editbam.mawk',
    "bam2csv": 'bam2csv',
    "genome_split_path": genome_split_path # the path to the folder with chrom-split genomes (chr1.fa..)
}
HDR_master(mut_file, bam_file=bam_file, chrom=chrom, threads=threads, HDR_config=HDR_config)

Importing /Users/mahtin/Dropbox/Icke/Work/somVar/HDRtest/data/002.csv for HDR detection
22:04:42 : Starting HDR analysis of /Users/mahtin/Dropbox/Icke/Work/somVar/HDRtest/data/002.csv on chr1 using 8 cores. [MIN_SIM=0.5, PAD=100]
22:04:42 : Using 4 cores for hotspot detection on chr1
22:04:47 : Detected 26 putative HDR lanes in /Users/mahtin/Dropbox/Icke/Work/somVar/HDRtest/data/002.bam .
22:04:47 : Using 8 cores for filtering HDR-rich mutations
22:04:47 : Found a total of 14 HDR-rich mutations
22:04:47 : Using 8 cores for HDR computation


,Chr,Start,End,Ref,Alt,Gene,HDRcand,HDRcount,HDRinfo
0,chr1,3632268,3632268,G,C,WRAP73,1,0,no HDR in vincinity
1,chr1,4712657,4712657,G,A,AJAP1,1,0,no HDR in vincinity
2,chr1,23417366,23417366,T,C,TCEA3,1,0,no HDR in vincinity
3,chr1,39511892,39511892,G,A,BMP8A,2,0,no similarity in HDR-pattern
4,chr1,39769776,39769776,C,T,OXCT2,2,1,∆41<Ref:100%(50)><Alt:100%(14)>
5,chr1,109192794,109192794,A,C,KIAA1324,2,1,∆88<Ref:100%(8)><Alt:80%(10)>
6,chr1,117118132,117118132,C,T,TRIM45,1,0,no HDR in vincinity
7,chr1,146135471,146135471,T,C,NBPF10,1,0,no HDR in vincinity
8,chr1,146164829,146164829,G,A,NOTCH2NL,1,0,no HDR in vincinity
9,chr1,152155775,152155775,G,T,RPTN,7,0,no similarity in HDR-pattern


In [ ]:
ls data

In [ ]:

home = '/Users/mahtin'
path = '/Users/mahtin/Dropbox/Icke/Work/somVar/HDRtest'
bam_file = f'{path}/data/002.bam '
chrom = 'chr1'
# chrom = ''
pileup_file = f'{path}/data/002.pileup'
threads = 8
mut_file = f'{path}/data/002.csv'
mut_df = pd.read_csv(mut_file, sep='\t').loc[:, [
        'Chr', 'Start', 'End', 'Ref', 'Alt', 'Gene']]
# make Chr column categorical for sorting .. and sort
chrom_list = [f"chr{i}" for i in range(23)] + ['chrX', 'chrY']
mut_df['Chr'] = pd.Categorical(mut_df['Chr'], chrom_list)
mut_df = mut_df.sort_values(['Chr', 'Start'])
mut_df = mut_df.query('Chr == @chrom')
mut_df
mut_split = np.array_split(mut_df, 4)

In [ ]:
mut_df = mut_split[0]

In [ ]:
anno_df = mut_df.sort_values(['Chr', 'Start']).iloc[0:,:5]
anno_df

In [ ]:
def reduce_regions(df, padding):
    '''
    takes a mutation list and returns a region list using padding
    overlapping regions are reduced to one using the gap strategy
    '''

    df = df.sort_values('Start')
    df['Start'] = df['Start'] - padding
    df['End'] = df['End'] + padding
    # find the break points
    # if Start is greater than previous End (using shift), this is a gap --> df['gap'] = 1
    df['gap'] = df['Start'].gt(df['End'].shift()).astype('int')
    # id different reads according to gap
    # cumulative sum does not increase at df['gap'] == 0 and so these consecutive stretches are grouped together
    df['gap'] = df['gap'].cumsum()
    # groupby the coverage break group and condense individual coverage islands
    # agg has to contain the neccessary shared columns TransLength because it is needed for coverage computation
    df = df.groupby('gap').agg({'Chr':'first','Start': 'min', 'End':'max'})
    return df.reset_index('gap').drop(columns='gap')

In [ ]:
df = anno_df
anno_df
padding = 100
df['Start'] = df['Start'] - padding
df['End'] = df['End'] + padding
df['gap'] = df['Start'].gt(df['End'].shift()).astype('int')
df['gap'] = df['gap'].cumsum()
df = df.groupby('gap').agg({'Chr':'first','Start': 'min', 'End':'max'}).reset_index().drop(columns='gap')
df

In [ ]:
bed_df = reduce_regions(anno_df, 100)
bed_df